In [101]:
import glob, re, sys, os, pandas as pd

try:
    RES = 'HTL' ## three-letter code of selected ligand atom
    dlg_dir = '/home/lab83bio/Ricerca/Rev_Docking'
except:
    print('USAGE: ./get_atom_energy.py MY_RES_IN_LIGAND MY_DLG_DIRECTORY')
    
## print header of table
print('dlg file', 'Ligand', 
      'BC (kcal/mol)', 'Run', 'LC (kcal/mol)', 'Run', 
      'BCaa (kcal/mol)', 'Run', 'LCaa (kcal/mol)','Run', 
      'BCM (kcal/mol)', 'LCM (kcal/mol)', 
      'BCaaM (kcal/mol)', 'LCaaM (kcal/mol)',
      'LC', 'Num in LC', 
      sep = '\t',
      file=open(dlg_dir+dlg_dir.split('/')[-2]+'.tsv', 'w')) # tab headers

for dlg in glob.glob(dlg_dir+'/*.dlg'):
    try:
        tab, dock = [], {}
        for line in open(dlg):
            if line.startswith('DOCKED: MODEL'):
                model = int(line.split()[-1])
            elif line.startswith('DOCKED: ATOM'):
                number, atom, res, chain, x, y, z, vdw, Elec = line.split()[2:11]
                if res == RES:
                    dock.setdefault(model, []).append((float(vdw)+float(Elec)))
            elif 'RANKING' in line:
                tab.append(list(map(float,line.split()[:4])))

        df_dock = pd.DataFrame.from_dict(dock, orient='index')
        df_dock['vdw+Elec'] = df_dock.sum(axis=1)

        df = pd.DataFrame(tab, columns = ['rank', 'subrank', 'run', 'energy'], dtype = int)
        df.set_index(['rank','run'], inplace = True)
        df = df.join(df_dock['vdw+Elec'], on='run').round(2)
        bo = df.groupby('rank')
        bo.idxmin().at[1,'vdw+Elec'][1]
        best_energy = (df.loc[1].iloc[0]['energy'], 
                       df.loc[1]['energy'].idxmin())
        largest_energy = (df.loc[df['subrank'].idxmax()[0]].iloc[0]['energy'], 
                          df.loc[df['subrank'].idxmax()[0]]['energy'].idxmin())
        best_res = (df.loc[1].iloc[0]['vdw+Elec'], 
                    df.loc[1]['subrank'].idxmin())
        largest_res = (df.loc[df['subrank'].idxmax()[0]].iloc[0]['vdw+Elec'], 
                       df.loc[df['subrank'].idxmax()[0]]['subrank'].idxmin())
        mean_largest = round(df.loc[df['subrank'].idxmax()[0]]['energy'].mean(),2)
        mean_best = round(df['energy'][1].mean(),2)
        best_largest_res = (df.loc[df['subrank'].idxmax()[0]]['vdw+Elec'].min(), 
                            df.loc[df['subrank'].idxmax()[0]]['vdw+Elec'].idxmin())
        mean_largest_res = round(df.loc[df['subrank'].idxmax()[0]]['vdw+Elec'].mean(),2)
        mean_best_res = round(df['vdw+Elec'][1].mean(),2)
        avg = round(df['vdw+Elec'].mean(),2)
        bestbest = (df['vdw+Elec'].min(), 
                    df['vdw+Elec'].idxmin()[1])
        largest_cluster = df['subrank'].idxmax()[0]
        largest_cluster_num = df['subrank'].max()
        print(os.path.basename(dlg), RES, 
              *best_energy, *largest_energy, 
              *best_res, *largest_res, 
              mean_best, mean_largest,
              mean_best_res, mean_largest_res,
              largest_cluster, largest_cluster_num, 
              sep = '\t', file=open(dlg_dir+dlg_dir.split('/')[-2]+'.tsv', 'a'))
    except:
        print(os.path.basename(dlg), sep = '\t', file=open(dlg_dir+dlg_dir.split('/')[-2]+'.tsv', 'a'))

In [97]:
largest_cluster_num

38

In [74]:
df

subrank  energy  vdw+Elec
rank run                           
1    29         1  -14.12     -6.63
     41         2  -13.93     -6.36
     33         3  -13.86     -6.07
     40         4  -13.66     -5.80
     46         5  -13.61     -6.06
     38         6  -13.60     -6.73
     23         7  -13.59     -6.00
     7          8  -13.46     -6.36
     11         9  -13.40     -5.98
     10        10  -13.38     -5.76
     36        11  -13.32     -5.85
     44        12  -13.17     -6.05
     30        13  -13.15     -5.51
     5         14  -13.13     -5.25
     9         15  -13.11     -6.53
     28        16  -13.08     -5.72
     32        17  -13.06     -5.45
     45        18  -12.92     -5.10
     21        19  -12.91     -5.15
     17        20  -12.90     -5.31
     24        21  -12.90     -5.89
     22        22  -12.85     -5.50
     2         23  -12.81     -4.98
     43        24  -12.78     -5.63
     14        25  -12.78     -5.11
     31        26  -12.68     -5.37
     18        27  -12.61     -6.36
     34        28  -12.57     -5.37
     19        29  -12.50     -4.73
     48        30  -12.47     -5.28
     50        31  -12.46     -5.66
     8         32  -12.35     -5.59
     16        33  -12.27     -6.08
     27        34  -12.26     -5.35
     1         35  -11.89     -5.25
     25        36  -11.79     -6.14
     12        37  -11.73     -5.60
     20        38  -10.99     -5.82
2    42         1  -13.96     -6.41
     6          2  -13.71     -6.13
     15         3  -13.59     -6.25
     47         4  -13.34     -5.82
     49         5  -13.33     -6.39
     35         6  -13.02     -5.61
     13         7  -12.72     -4.95
     26         8  -12.66     -5.42
     4          9  -12.45     -5.38
     37        10  -12.04     -5.21
     39        11  -11.94     -5.97
3    3          1   -6.68     -5.10